In [43]:
import sys
from sklearn.metrics import mean_absolute_error
from dataloader import APPLIANCE_ORDER, get_train_test
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable

cuda_av = False
if torch.cuda.is_available():
    cuda_av = True

torch.manual_seed(0)
np.random.seed(0)
tensor = np.load("../2015-5appliances-missing-true-agg.npy")


In [44]:
train, test = get_train_test(2, 5, 0)

In [45]:
new_dataset = np.vstack([tensor, train])

In [53]:

class CustomRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional):
        super(CustomRNN, self).__init__()
        torch.manual_seed(0)

        if bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        if cell_type == "RNN":
            self.rnn = nn.RNN(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        elif cell_type == "GRU":
            self.rnn = nn.GRU(input_size=1, hidden_size=hidden_size,
                              num_layers=num_layers, batch_first=True,
                              bidirectional=bidirectional)
        else:
            self.rnn = nn.LSTM(input_size=1, hidden_size=hidden_size,
                               num_layers=num_layers, batch_first=True,
                               bidirectional=bidirectional)

        self.linear = nn.Linear(hidden_size * self.num_directions, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        pred, hidden = self.rnn(x, None)
        pred = self.linear(pred).view(pred.data.shape[0], -1, 1)
        # pred = self.act(pred)
        # pred = torch.clamp(pred, min=0.)
        pred = self.act(pred)
        pred = torch.min(pred, x)
        return pred


class AppliancesRNN(nn.Module):
    def __init__(self, cell_type, hidden_size, num_layers, bidirectional, num_appliance):
        super(AppliancesRNN, self).__init__()
        self.num_appliance = num_appliance
        self.preds = {}
        self.order = ORDER
        for appliance in range(self.num_appliance):
            if cuda_av:
                setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type,
                                                                       hidden_size,
                                                                       num_layers,
                                                                       bidirectional).cuda())
            else:
                setattr(self, "Appliance_" + str(appliance), CustomRNN(cell_type,
                                                                       hidden_size,
                                                                       num_layers,
                                                                       bidirectional))

    def forward(self, *args):
        agg_current = args[0]
        flag = False
        if np.random.random() > args[1]:
            flag = True
            # print("Subtracting prediction")
        else:
            pass
            # print("Subtracting true")
        for appliance in range(self.num_appliance):
            # print(agg_current.mean().data[0])
            # print (appliance)
            # print (self.order[appliance])
            # print (args[2+appliance])
            # print(getattr(self, "Appliance_" + str(appliance)))
            self.preds[appliance] = getattr(self, "Appliance_" + str(appliance))(agg_current)
            if flag:
                agg_current = agg_current - self.preds[appliance]
            else:
                agg_current = agg_current - args[2 + appliance]

        return torch.cat([self.preds[a] for a in range(self.num_appliance)])


In [50]:
new_aggregate = new_dataset[:, 0, :, :].reshape(-1, 24, 1)
#test_aggregate = test[:, 0, :, :].reshape(-1, 24, 1)
ORDER = ['mw', 'dw', 'dr', 'fridge', 'hvac']

out_train = [None for temp in range(len(ORDER))]
for a_num, appliance in enumerate(ORDER):
    out_train[a_num] = Variable(
        torch.Tensor(new_dataset[:, APPLIANCE_ORDER.index(appliance), :, :].reshape((new_aggregate.shape[0], -1, 1))))
    if cuda_av:
        out_train[a_num] = out_train[a_num].cuda()

In [55]:
loss_func = nn.L1Loss()
a = AppliancesRNN('GRU', 20, 2, True, len(ORDER))
# prevent negative
for param in a.parameters():
    param.data = param.data.abs()
#print(a)
if cuda_av:
    a = a.cuda()
    loss_func = loss_func.cuda()
optimizer = torch.optim.Adam(a.parameters(), lr=1)

inp = Variable(torch.Tensor(new_aggregate.reshape((new_aggregate.shape[0], -1, 1))).type(torch.FloatTensor),
               requires_grad=True)

In [ ]:
for t in range(10):
    inp = Variable(torch.Tensor(new_aggregate), requires_grad=True)
    out = torch.cat([out_train[appliance_num] for appliance_num, appliance in enumerate(ORDER)])
    if cuda_av:
        inp = inp.cuda()
        out = out.cuda()

    params = [inp, 0]
    for a_num, appliance in enumerate(ORDER):
        params.append(out_train[a_num])
    # print(params)
    pred = a(*params)

    optimizer.zero_grad()
    mask = out > -1* 1e8
    loss = loss_func(pred[mask], out[mask])
    if t % 1 == 0:
        print(t, loss.data[0])

    loss.backward()
    optimizer.step()